In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [3]:
formulas = pd.read_csv("../data/raw/im2latex_formulas.norm.csv")
formulas.head()

,formulas
0,\int _ { - \epsilon } ^ { \infty } d l \: \mat...
1,d s ^ { 2 } = ( 1 - { \frac { q c o s \theta }...
2,\widetilde \gamma _ { \mathrm { h o p f } } \s...
3,"( { \cal L } _ { a } g ) _ { i j } = 0 , \ \ \..."
4,S _ { s t a t } = 2 \pi \sqrt { N _ { 5 } ^ { ...


In [4]:
sample = formulas.iloc[8332]

In [5]:
print(sample.formulas)
# print(sample.image)

\langle f _ { \lambda } f _ { \lambda ^ { \prime } } \rangle \propto \int d ^ { 2 } x \sqrt { - g } e ^ { - 2 \phi } f _ { \lambda } f _ { \lambda ^ { \prime } } = \delta _ { \lambda , \lambda ^ { \prime } } \quad .


In [6]:
from IPython.display import display, Math, Latex
display(Math(rf'{sample.formulas}'))
display(Math(rf'{formulas.loc[234].formulas}'))
display(Math('g \\approx 3 - \\sqrt 3 - 0 . 9 1 7 7 f _ { 0 } ^ { 2 } \\; .'))
display(Math('g \\approx  3 - \\sqrt  3 -  0  .   9  1  7  7 f _ {  0 } ^ {  2 } \\;  . '))


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [7]:
formulas.loc[234].formulas

'g \\approx 3 - \\sqrt 3 - 0 . 9 1 7 7 f _ { 0 } ^ { 2 } \\; .'

In [8]:
formulas.shape

(102863, 1)

In [9]:
formulas = pd.DataFrame(formulas)

In [10]:
formulas.head()

,formulas
0,\int _ { - \epsilon } ^ { \infty } d l \: \mat...
1,d s ^ { 2 } = ( 1 - { \frac { q c o s \theta }...
2,\widetilde \gamma _ { \mathrm { h o p f } } \s...
3,"( { \cal L } _ { a } g ) _ { i j } = 0 , \ \ \..."
4,S _ { s t a t } = 2 \pi \sqrt { N _ { 5 } ^ { ...


In [11]:
re.sub('(\d)', r' \1 ', "encode25decode")

'encode 2  5 decode'

In [12]:
formulas.loc[234, "formulas"]

'g \\approx 3 - \\sqrt 3 - 0 . 9 1 7 7 f _ { 0 } ^ { 2 } \\; .'

In [13]:
formulas["formulas"] = formulas["formulas"].apply(lambda x: x.replace(".", " . "))
formulas["formulas"] = formulas["formulas"].apply(lambda x: re.sub("(\d)", r" \1", x))

In [14]:
formulas.loc[234, "formulas"]

'g \\approx  3 - \\sqrt  3 -  0  .   9  1  7  7 f _ {  0 } ^ {  2 } \\;  . '

In [15]:
formulas["formula_tokenized"] = formulas["formulas"].apply(lambda x: x.strip().split())

In [16]:
formulas.loc[234, "formula_tokenized"]

['g',
 '\\approx',
 '3',
 '-',
 '\\sqrt',
 '3',
 '-',
 '0',
 '.',
 '9',
 '1',
 '7',
 '7',
 'f',
 '_',
 '{',
 '0',
 '}',
 '^',
 '{',
 '2',
 '}',
 '\\;',
 '.']

In [17]:
formulas["formula_len"] = formulas["formula_tokenized"].apply(lambda x: len(x))

In [18]:
formulas.formula_len.describe()

count    102863.000000
mean         65.601509
std          42.956291
min           1.000000
25%          38.000000
50%          55.000000
75%          82.000000
max        1051.000000
Name: formula_len, dtype: float64

In [19]:
np.quantile(formulas.formula_len, q=.999)

415.1380000000063

[TODO] Set max_len to 512

In [20]:
from sklearn.feature_extraction.text import CountVectorizer

In [21]:
vectorizer = CountVectorizer()
vectorizer.fit_transform(formulas["formulas"])
len(vectorizer.get_feature_names_out())

371

In [22]:
vectorizer.get_feature_names_out()

array(['aa', 'acute', 'ae', 'aleph', 'alpha', 'amalg', 'approx', 'array',
       'arraycolsep', 'arraystretch', 'arrowvert', 'ast', 'asymp', 'atop',
       'atopwithdelims', 'backslash', 'bar', 'begin', 'beta', 'bf', 'big',
       'bigcap', 'bigcirc', 'bigcup', 'bigg', 'biggl', 'biggm', 'biggr',
       'bigl', 'bigm', 'bigoplus', 'bigotimes', 'bigr', 'bigskip',
       'bigsqcup', 'bigtriangledown', 'bigtriangleup', 'bigvee',
       'bigwedge', 'binom', 'bmod', 'boldmath', 'bot', 'brace', 'brack',
       'breve', 'buildrel', 'bullet', 'cal', 'cap', 'cdot', 'cdotp',
       'cdots', 'check', 'chi', 'circ', 'circle', 'cite', 'cline', 'cm',
       'colon', 'cong', 'coprod', 'crcr', 'cup', 'dag', 'dagger', 'ddag',
       'ddagger', 'ddot', 'ddots', 'def', 'delta', 'diamond',
       'diamondsuit', 'displaystyle', 'do', 'dot', 'doteq', 'dotfill',
       'dots', 'downarrow', 'ell', 'em', 'emph', 'emptyset', 'end',
       'enskip', 'enspace', 'ensuremath', 'epsilon', 'equiv', 'eta',
       'ever

In [23]:
words = formulas["formula_tokenized"].tolist()
len(words)

102863

In [81]:
from collections import Counter

vocab = Counter([x for sublist in words for x in sublist])

In [82]:
len(vocab)

562

In [83]:
vocab = sorted(list(vocab.keys()))
vocab = ["<UNK>", "<SOS>", "<PAD>", "<EOS>"] + vocab
vocab = {k: i for i, k in enumerate(vocab)}
vocab['a']

531

In [84]:
import json

In [85]:
with open("../data/interim/token2id.json", "w") as f:
    json.dump(vocab, f)

In [86]:
inverse_vocab = {v: k for k, v in vocab.items()}

In [87]:
with open("../data/interim/id2token.json", "w") as f:
    json.dump(inverse_vocab, f)